# Prepare data

*By Joe Marcus*

Here I prepare two dataset for gom. For both datasets we use a set of ~16K genes defined in Dey et al. 2017 that pass QC filters ...

* *Dataset 1* - cell-free RNA data for 48 samples from Koh et al. 2017
* *Dataset 2* - the intersection of these 48 samples with ~10k individuals from GTEx

## Configuration

In [1]:
suppressMessages(library(data.table))
suppressMessages(library(readr))
suppressMessages(library(dplyr))
suppressMessages(library(SummarizedExperiment))

## Paths

In [52]:
mod_cfrna_path <- '/project/jnovembre/data/external_public/mod_cfrna'
koh_meta_path <- paste0(mod_cfrna_path, '/koh_et_al_2014/koh_et_al_covariates.csv')
gtex_meta_path <- paste0(mod_cfrna_path, '/gtex/SRP012682.tsv')

meta_path <- '../output/meta/koh_gtex_meta.rds'
koh_count_path <- '../output/counts/koh_count_dey_genes.rds'
koh_gtex_count_path <- '../output/counts/koh_gtex_count_dey_genes.rds'

## Genes

Gene table as defined by `recount2`

In [3]:
rse_genes_obj <- load(paste0(mod_cfrna_path, '/koh_et_al_2014/rse_gene.Rdata'))
rse_genes <- get(rse_genes_obj)
rowRanges(rse_genes)

GRanges object with 58037 ranges and 3 metadata columns:
                     seqnames                 ranges strand |
                        <Rle>              <IRanges>  <Rle> |
  ENSG00000000003.14     chrX [100627109, 100639991]      - |
   ENSG00000000005.5     chrX [100584802, 100599885]      + |
  ENSG00000000419.12    chr20 [ 50934867,  50958555]      - |
  ENSG00000000457.13     chr1 [169849631, 169894267]      - |
  ENSG00000000460.16     chr1 [169662007, 169854080]      + |
                 ...      ...                    ...    ... .
   ENSG00000283695.1    chr19 [ 52865369,  52865429]      - |
   ENSG00000283696.1     chr1 [161399409, 161422424]      + |
   ENSG00000283697.1     chrX [149548210, 149549852]      - |
   ENSG00000283698.1     chr2 [112439312, 112469687]      - |
   ENSG00000283699.1    chr10 [ 12653138,  12653197]      - |
                                gene_id bp_length          symbol
                            <character> <integer> <CharacterList>
  ENS

remove "." to intersect with Dey et al. gene ids ...

In [4]:
gene_ids <- gsub('\\..*', '', rowRanges(rse_gene)$gene_id)
gene_symbols <- unname(as.vector(rowRanges(rse_gene)$symbol))
rb_mt_idx <- which(!is.na(match(substring(gene_symbols, 1, 2), c("RP", "MT", "RN"))))
rb_mt_ids <- gene_ids[rb_mt_idx]

head(gene_ids)
length(gene_ids)
length(rb_mt_ids)

[1] "ENSG00000000003" "ENSG00000000005" "ENSG00000000419" "ENSG00000000457"
[5] "ENSG00000000460" "ENSG00000000938"

[1] 58037

[1] 433

### Dey et al. genes

> We analyzed 16,069 genes that satisfied filters (e.g. exceeding certain minimum expression levels) that were used during eQTL analyses by the GTEx project

In [5]:
gtex_fil_genes <- read.table(paste0(mod_cfrna_path, '/gtex/gene_names_GTEX_V6.txt'), 
                             header=TRUE, stringsAsFactors=FALSE)$cis_gene_names
gtex_fil_genes <- gsub('\\..*', '', gtex_fil_genes)

head(gtex_fil_genes)
length(gtex_fil_genes)

[1] "ENSG00000000419" "ENSG00000000457" "ENSG00000000460" "ENSG00000000938"
[5] "ENSG00000000971" "ENSG00000001036"

[1] 16069

remove ribosomal and mt rnas which are highly expressed and could add a lot of noise ...

In [6]:
gtex_fil_genes <- gtex_fil_genes[!(gtex_fil_genes %in% rb_mt_ids)]

head(gtex_fil_genes)
length(gtex_fil_genes)

[1] "ENSG00000000419" "ENSG00000000457" "ENSG00000000460" "ENSG00000000938"
[5] "ENSG00000000971" "ENSG00000001036"

[1] 15799

This removed about ~300 genes

## Count data

Data downloaded from `recount2`

### Koh et al. 2014

In [7]:
koh_count_mat <- as.matrix(fread(paste0(mod_cfrna_path, '/koh_et_al_2014/counts_gene.tsv')))
rownames(koh_count_mat) <- gene_ids
dim(koh_count_mat)

[1] 58037    48

Lets keep only the genes that are in the filtered set defined by Dey et al. 2017 ...

In [8]:
koh_count_mat <- koh_count_mat[rownames(koh_count_mat) %in% gtex_fil_genes, ]
dim(koh_count_mat)

[1] 15322    48

*???some genes in the Dey et al. set are missing from Koh et al and not sure why???*

### GTEx

In [9]:
gtex_count_mat <- as.matrix(fread(paste0(mod_cfrna_path, '/gtex/counts_gene.tsv')))

Read 58037 rows and 9662 (of 9662) columns from 2.146 GB file in 00:00:55


In [10]:
rownames(gtex_count_mat) <- gene_ids
dim(gtex_count_mat)

[1] 58037  9662

In [11]:
gtex_count_mat <- gtex_count_mat[rownames(gtex_count_mat) %in% rownames(koh_count_mat), ]
dim(gtex_count_mat)

[1] 15322  9662

bind the columns of the Koh et al counts and GTEx counts to create a single dataset ...

In [12]:
koh_gtex_count_mat <- cbind(gtex_count_mat, koh_count_mat[rownames(gtex_count_mat),])
dim(koh_gtex_count_mat)

[1] 15322  9710

## Meta data

In [51]:
koh_meta_df <- read_csv(koh_meta_path)
gtex_meta_df <- read_tsv(gtex_meta_path) %>% select(experiment, run, smtsd, smts) 
colnames(gtex_meta_df) <- c('id', 'run_id', 'iid', 'label')
head(koh_meta_df)
head(gtex_meta_df)
nrow(koh_meta_df)
nrow(gtex_meta_df)

koh_gtex_meta_df <- bind_rows(koh_meta_df, gtex_meta_df)
dim(koh_gtex_meta_df)

Parsed with column specification:
cols(
  id = col_character(),
  run_id = col_character(),
  iid = col_character(),
  label = col_character()
)
Parsed with column specification:
cols(
  .default = col_integer(),
  project = col_character(),
  sample = col_character(),
  experiment = col_character(),
  run = col_character(),
  proportion_of_reads_reported_by_sra_downloaded = col_double(),
  paired_end = col_logical(),
  sra_misreported_paired_end = col_logical(),
  auc = col_double(),
  sharq_beta_tissue = col_character(),
  sharq_beta_cell_type = col_character(),
  biosample_submission_date = col_character(),
  biosample_publication_date = col_character(),
  biosample_update_date = col_character(),
  geo_accession = col_character(),
  bigwig_file = col_character(),
  sampid = col_character(),
  smcenter = col_character(),
  smpthnts = col_character(),
  smrin = col_double(),
  smts = col_character()
  # ... with 33 more columns
)
See spec(...) for full column specifications.


id,run_id,iid,label
SRX550521,SRR1296083,Control 4,Not pregnant
SRX550520,SRR1296082,Control 3,Not pregnant
SRX550519,SRR1296081,Control 2,Not pregnant
SRX550518,SRR1296080,Control 1,Not pregnant
SRX550500,SRR1296062,Patient 36,Trimester 3
SRX550493,SRR1296055,Patient 32,Post-Partum


id,run_id,iid,label
SRX222703,SRR660824,Lung,Lung
SRX1152700,SRR2166176,Brain - Cerebellar Hemisphere,Brain
SRX199032,SRR606939,Heart - Left Ventricle,Heart
SRX1153642,SRR2167642,Brain - Cerebellar Hemisphere,Brain
SRX1152198,SRR2165473,Skin - Sun Exposed (Lower leg),Skin
SRX199539,SRR603858,Lung,Lung


[1] 48

[1] 9662

[1] 9710    4

## Save meta data

In [53]:
saveRDS(koh_gtex_meta_df, meta_path)

## Save count matricies

*Dataset 1*

In [13]:
saveRDS(koh_count_mat, koh_count_path)

*Dataset 2*

In [14]:
saveRDS(koh_gtex_count_mat, koh_gtex_count_path)